# RDF-   NLS -  Encyclopaedia Britannica

This notebook is going to create the RDF triples to generate our RDLIB GRAPH

For each postprocess edition dataframe that we got from **Merging_EB_Terms.ipynb** (e.g. results_eb_1_edition_dataframe, results_eb_2_edition_dataframe, etc) we are going to add the information from the dataframe that we got from **Metadata_EB.ipynb** (metadata_eb_dataframe). 

The idea is to have per edition dataframe (and also supplement dataframe), all the information (which currently is splitted across several dataframes) in one. 


This notebook will store the final dataframes in results_NLS directory, and their name schema will be **final_eb_< NUM_EDITION >_dataframe**.

Per entry in these new dataframes we will have the following columns (see an example of one entry of the first edition):

- MMSID:                                              
- editionTitle:                          First edition, 1771, Volume 1, A-B
- editor:                                                  Smellie, William
- editor_date:                                                   1740-1795
- genre:                                                       encyclopedia
- language:                                                             eng
- termsOfAddress:                                                       NaN
- numberOfPages:                                                        832
- physicalDescription:               3 v., 160 plates : ill. ; 26 cm. (4to)
- place:                                                         Edinburgh
- publisher:              Printed for A. Bell and C. Macfarquhar; and so...
- referencedBy:           [Alston, R.C.  Engl. language III, 560, ESTC T...
- shelfLocator:                                                        EB.1
- editionSubTitle:        Illustrated with one hundred and sixty copperp...
- volumeTitle:            Encyclopaedia Britannica; or, A dictionary of ...
- year:                                                                1771
- volumeId:                                                       144133901
- metsXML:                                               144133901-mets.xml
- permanentURL:                            https://digital.nls.uk/144133901
- publisherPersons:                     [C. Macfarquhar, Colin Macfarquhar]
- volumeNum:                                                              1
- letters:                                                              A-B
- part:                                                                   0
- editionNum:                                                             1
- supplementTitle:                                                         
- supplementSubTitle:                                                      
- supplementsTo:                                                         []
- numberOfVolumes:                                                        6
- term:                                                                  OR
- definition:             A NEW A D I C T I A A, the name of several riv...
- relatedTerms:                                                          []
- header:                                           EncyclopaediaBritannica
- startsAt:                                                              15
- endsAt:                                                                15
- numberOfTerms:                                                         22
- numberOfWords:                                                         54
- positionPage:                                                           0
- typeTerm:                                                         Article
- altoXML:                                  144133901/alto/188082904.34.xml

### Loading the necessary libraries

In [1]:
import yaml
import numpy as np
import collections
import string
import copy

In [2]:
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize
from difflib import SequenceMatcher

### Functions

### 1. Loading the final dataframe

In [3]:
df= pd.read_json('../../results_NLS/final_eb_1_dataframe', orient="index") 

In [4]:
df=df.fillna(0)

Lets get the first element of the "final_eb_1_dataframe" dataframe and extract the information of the 1st Edition class.

In [5]:
df_1771=df[df['year'] == 1771].reset_index(drop=True)

In [6]:
data = df_1771.loc[0]

### 2. Create a Graph and import the information of the 1st Edition class to it.  

In [7]:
from rdflib import Graph, URIRef, Literal, Namespace, XSD
from rdflib.namespace import RDF, RDFS


# Create a Graph
g = Graph()

g.namespace_manager.bind('eb', Namespace("https://w3id.org/eb#"), override="False")
eb = Namespace("https://w3id.org/eb#")

editionFirst = URIRef("https://w3id.org/eb/i/Edition/"+str(data["MMSID"]))

#### Edition Information

g.add((editionFirst, RDF.type, eb.Edition))
g.add((editionFirst, eb.number, Literal(data["editionNum"], datatype=XSD.integer)))
g.add((editionFirst, eb.title, Literal("First Edition, 1771", datatype=XSD.string)))
g.add((editionFirst, eb.subtitle, Literal(data["editionSubTitle"], datatype=XSD.string)))
g.add((editionFirst, eb.publicationYear, Literal(data["year"], datatype=XSD.integer)))
g.add((editionFirst, eb.place, Literal(data["place"], datatype=XSD.string)))
g.add((editionFirst, eb.mmsid, Literal(data["MMSID"], datatype=XSD.int)))
g.add((editionFirst, eb.physicalDescription, Literal(data["physicalDescription"], datatype=XSD.string)))
g.add((editionFirst, eb.genre, Literal(data["genre"], datatype=XSD.string)))
g.add((editionFirst, eb.language, Literal(data["language"], datatype=XSD.string)))
g.add((editionFirst, eb.shelfLocator, Literal(data["shelfLocator"], datatype=XSD.string)))
g.add((editionFirst, eb.numberOfVolumes, Literal(data["numberOfVolumes"], datatype=XSD.integer)))

#### Editor 

name=data["editor"].replace(" ", "")
editorFirst = URIRef("https://w3id.org/eb/i/Person/"+str(name))
g.add((editorFirst, RDF.type, eb.Person))
g.add((editorFirst, eb.name, Literal(data["editor"], datatype=XSD.string)))

if data["editor_date"]!=0:
    tmpDate=data["editor_date"].split("-")
    birthDate=tmpDate[0]
    deathDate=tmpDate[1]
    g.add((editorFirst, eb.birthDate, Literal(birthDate, datatype=XSD.dateTime)))
    g.add((editorFirst, eb.deathDate, Literal(deathDate, datatype=XSD.dateTime)))
    
if data["termsOfAddress"] != 0:
    g.add((editorFirst, eb.termsOfAddress, Literal(data["termsOfAddress"], datatype=XSD.string)))

g.add((editionFirst, eb.editor, editorFirst))

#### Publishers Persons 

#This was the result to pass entity recognition to publisher

if data["publisherPersons"] != 0:
    publisherPersons=name=data["publisherPersons"]
    for p in publisherPersons: 
        name=p.replace(" ", "")
        publisher = URIRef("https://w3id.org/eb/i/Person/"+name)
        g.add((publisher, RDF.type, eb.Person))
        g.add((publisher, eb.name, Literal(p, datatype=XSD.string)))
        g.add((editionFirst, eb.publisher, publisher))
        
#### Is Referenced by  

if data["referencedBy"] != 0:
    references=data["referencedBy"]
    for r in references: 
        name=r.replace(" ", "")
        book = URIRef("https://w3id.org/eb/i/Book/"+name)
        g.add((book, RDF.type, eb.Book))
        g.add((book, eb.title, Literal(r, datatype=XSD.string)))
        g.add((editionFirst, eb.referencedBy, book))
        
### Volumes of 1171

list_vols = df_1771["volumeNum"].unique()
print("list_vols %s" % (list_vols))
for v in range(0,len(list_vols)):
    df_1771_vl=df_1771[df_1771["volumeNum"] == list_vols[v]].reset_index(drop=True)
    volume_data=df_1771_vl.loc[0]
    volume_id=volume_data["volumeId"]
    volume = URIRef("https://w3id.org/eb/i/Volume/"+str(volume_id))
    g.add((volume, RDF.type, eb.Volume))
    g.add((volume, eb.number, Literal(volume_data["volumeNum"], datatype=XSD.integer)))
    g.add((volume, eb.letters, Literal(volume_data["letters"], datatype=XSD.string)))
    g.add((volume, eb.volumeId, Literal(volume_data["volumeId"], datatype=XSD.int)))
    g.add((volume, eb.title, Literal(volume_data["volumeTitle"], datatype=XSD.string)))
    if volume_data["part"]!=0:
        g.add((volume, eb.part, Literal(volume_data["part"], datatype=XSD.string)))
    
    g.add((volume, eb.metsXML, Literal(volume_data["metsXML"], datatype=XSD.string)))
    g.add((volume, eb.permanentURL, Literal(volume_data["permanentURL"], datatype=XSD.string)))
    g.add((volume, eb.numberOfPages, Literal(volume_data["numberOfPages"], datatype=XSD.string)))
    
    g.add((editionFirst, eb.hasPart, volume))
    
    df_by_term=df_1771_vl.groupby(['term'])["term"].count().reset_index(name='counts')
    for t_index in range(0, len(df_by_term)):
        term=df_by_term.loc[t_index]["term"]
        count=df_by_term.loc[t_index]["counts"]
        if count == 1:
            df_entry= df_1771_vl[df_1771_vl["term"] == term].reset_index(drop=True).loc[0]
            if df_entry["typeTerm"] == "Article" :
                term= URIRef("https://w3id.org/eb/i/Article/"+str(df_entry["MMSID"])+"_"+str(df_entry["volumeId"])+"_"+term+"_"+str(count))
                g.add((term, RDF.type, eb.Article))
            elif df_entry["typeTerm"] == "Topic" :
                term= URIRef("https://w3id.org/eb/i/Topic/"+str(df_entry["MMSID"])+"_"+str(df_entry["volumeId"])+"_"+term+"_"+str(count))
                g.add((term, RDF.type, eb.Topic))
            else:
                pass  
            g.add((term, eb.name, Literal(term, datatype=XSD.string)))
            g.add((term, eb.definition, Literal(df_entry["definition"], datatype=XSD.string)))
            g.add((term, eb.positionPage, Literal(df_entry["positionPage"], datatype=XSD.int)))
            g.add((term, eb.numberOfWords, Literal(df_entry["numberOfWords"], datatype=XSD.int)))
            g.add((volume, eb.hasPart, term))
            
            ## startsAt
            page_startsAt= URIRef("https://w3id.org/eb/i/Page/"+ str(df_entry["MMSID"])+"_"+str(df_entry["volumeId"])+"_"+str(df_entry["startsAt"]))
            g.add((page_startsAt, RDF.type, eb.Page))
            g.add((page_startsAt, eb.number, Literal(df_entry["startsAt"], datatype=XSD.int)))
            g.add((page_startsAt, eb.header, Literal(df_entry["startsAt"], datatype=XSD.int)))
            g.add((volume, eb.hasPart, page_startsAt))
            g.add((term, eb.startsAt, page_startsAt))
            g.add((page_startsAt, eb.hasPart, term))
            
            ## endsAt
            page_endsAt= URIRef("https://w3id.org/eb/i/Page/"+ str(df_entry["MMSID"])+"_"+str(df_entry["volumeId"])+"_"+str(df_entry["endsAt"]))
            g.add((page_endsAt, RDF.type, eb.Page))
            g.add((page_endsAt, eb.number, Literal(df_entry["endsAt"], datatype=XSD.int)))
            g.add((page_endsAt, eb.header, Literal(df_entry["endsAt"], datatype=XSD.int)))
            g.add((volume, eb.hasPart, page_endsAt))
            g.add((term, eb.endsAt, page_endsAt))
            g.add((page_endsAt, eb.hasPart, term))
            
            
            
        else:
            pass
                
             
                          
# Save the Graph in the RDF Turtle format
g.serialize(format="turtle", destination="example.ttl")



list_vols [1 2 3]


List all the elements that we have added for the firstEdition

In [8]:
for s,p,o in g.triples((editionFirst, None, None)):
  print(s,p,o)

https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#subtitle Illustrated with one hundred and sixty copperplates
https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#place Edinburgh
https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#title First Edition, 1771
https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#publisher https://w3id.org/eb/i/Person/C.Macfarquhar
https://w3id.org/eb/i/Edition/992277653804341 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Edition
https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#editor https://w3id.org/eb/i/Person/Smellie,William
https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Volume/144133902
https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#publisher https://w3id.org/eb/i/Person/ColinMacfarquhar
https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#genre encyclopedia
https://w3id.org/eb/

In [9]:
for s,p,o in g.triples((editorFirst, None, None)):
  print(s,p,o)

https://w3id.org/eb/i/Person/Smellie,William https://w3id.org/eb#name Smellie, William
https://w3id.org/eb/i/Person/Smellie,William http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Person
https://w3id.org/eb/i/Person/Smellie,William https://w3id.org/eb#deathDate 1795
https://w3id.org/eb/i/Person/Smellie,William https://w3id.org/eb#birthDate 1740


In [10]:
g2 = Graph()
g2.parse("example.ttl", format="ttl") 

<Graph identifier=N1022f10c12e14f71b1a4c2e94c3bc64e (<class 'rdflib.graph.Graph'>)>

In [11]:
for s,p,o in g2.triples((editorFirst, None, None)):
  print(s,p,o)

https://w3id.org/eb/i/Person/Smellie,William https://w3id.org/eb#deathDate 1795
https://w3id.org/eb/i/Person/Smellie,William http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Person
https://w3id.org/eb/i/Person/Smellie,William https://w3id.org/eb#name Smellie, William
https://w3id.org/eb/i/Person/Smellie,William https://w3id.org/eb#birthDate 1740


In [12]:
for s,p,o in g2.triples((None, eb.publisher, None)):
  print(s,p,o)

https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#publisher https://w3id.org/eb/i/Person/ColinMacfarquhar
https://w3id.org/eb/i/Edition/992277653804341 https://w3id.org/eb#publisher https://w3id.org/eb/i/Person/C.Macfarquhar


In [15]:
#for s,p,o in g2.triples((None, eb.hasPart, None)):
#  print(s,p,o)

https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_SEMINATION_1
https://w3id.org/eb/i/Page/992277653804341_144133902_779 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_HETEROSCII_1
https://w3id.org/eb/i/Page/992277653804341_144133903_593 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_PRECIPITANT_1
https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_BREVIBUS_1
https://w3id.org/eb/i/Page/992277653804341_144133902_970 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_LEUCOIUM_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_SANICULA_1
https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_AMPHTHILL_1


https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Page/992277653804341_144133901_22
https://w3id.org/eb/i/Page/992277653804341_144133902_310 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_CORNAGE_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_MEDICAGO_1
https://w3id.org/eb/i/Page/992277653804341_144133902_598 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_FRATERNITY_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_WURTEMBURG_1
https://w3id.org/eb/i/Page/992277653804341_144133903_826 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_TUBERCLE_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_WOLFERDYKE_1
https://w3id.o

https://w3id.org/eb/i/Page/992277653804341_144133902_334 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_DAM_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_JAFFA_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_STRALSUND_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Topic/992277653804341_144133903_SIMPLE_1
https://w3id.org/eb/i/Page/992277653804341_144133903_747 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_SURCULUS_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Page/992277653804341_144133902_968
https://w3id.org/eb/i/Page/992277653804341_144133902_26 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_CAMERLINGO_1
https://w3id.org/eb/i/Page/9

https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Page/992277653804341_144133902_784
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_OMBRO_1
https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_BEAKED_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_TRAGIA_1
https://w3id.org/eb/i/Page/992277653804341_144133902_604 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_FRUMENTARII_1
https://w3id.org/eb/i/Page/992277653804341_144133903_808 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_TINUS_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_STRASBURG_1
https://w3id.org/eb/i/Volume/144133903 htt

https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_BOTTLE_1
https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_ACAMATOS_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_INTERREX_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_CLEOME_1
https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_ANASSA_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_CIMA_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_IMPROPRIATION_1
https://w3id.org/eb/i/Page/992277653804341_144133901_396 https://w3

...

https://w3id.org/eb/i/Page/992277653804341_144133903_197 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_MESQPTERYGIUS_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_CASSANO_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_CREDITON_1
https://w3id.org/eb/i/Page/992277653804341_144133903_726 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_SPATHA_1
https://w3id.org/eb/i/Page/992277653804341_144133902_1013 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_LUN_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_PARENZO_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_PAR_1
https://w3id.org

https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_BELIEF_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_FISH_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_REPULSION_1
https://w3id.org/eb/i/Page/992277653804341_144133902_825 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_JAMBA_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_GOTHIC_1
https://w3id.org/eb/i/Page/992277653804341_144133902_37 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_GANG_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_CORISPERMUM_1
https://w3id.org/eb/i/Page/99227765380

https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_GONORRHOEA_1
https://w3id.org/eb/i/Page/992277653804341_144133902_342 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_DEFAMATION_1
https://w3id.org/eb/i/Page/992277653804341_144133902_843 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_INSTANT_1
https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_AMMERGAW_1
https://w3id.org/eb/i/Page/992277653804341_144133902_834 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_IMBIBING_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_DARLINGTON_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_HEPTANGULAR_1


https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_ACOEMETAE_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_LAMIUM_1
https://w3id.org/eb/i/Page/992277653804341_144133903_620 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_REGULATOR_1
https://w3id.org/eb/i/Page/992277653804341_144133901_17 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_ABERBROTHOCK_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_STRATI_1
https://w3id.org/eb/i/Page/992277653804341_144133901_157 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_ALOETICS_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_RENIFORM_1
https:/

https://w3id.org/eb/i/Page/992277653804341_144133903_478 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_OGLIO_1
https://w3id.org/eb/i/Page/992277653804341_144133903_659 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_SCQTISTS_1
https://w3id.org/eb/i/Page/992277653804341_144133903_723 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_SOLUTION_1
https://w3id.org/eb/i/Volume/144133903 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133903_NEST_1
https://w3id.org/eb/i/Volume/144133902 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133902_FERRARA_1
https://w3id.org/eb/i/Volume/144133901 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_BALSARA_1
https://w3id.org/eb/i/Page/992277653804341_144133901_174 https://w3id.org/eb#hasPart https://w3id.org/eb/i/Article/992277653804341_144133901_ANACAMPSER

In [16]:
for s,p,o in g2.triples((None, None, eb.Topic)):
  print(s,p,o)

https://w3id.org/eb/i/Topic/992277653804341_144133903_ITASTAMI_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Topic
https://w3id.org/eb/i/Topic/992277653804341_144133902_DUBDUEHWW_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Topic
https://w3id.org/eb/i/Topic/992277653804341_144133902_GARDENING_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Topic
https://w3id.org/eb/i/Topic/992277653804341_144133902_GEOMETRY_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Topic
https://w3id.org/eb/i/Topic/992277653804341_144133902_GRAMMAR_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Topic
https://w3id.org/eb/i/Topic/992277653804341_144133903_PNEUMATICS_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Topic
https://w3id.org/eb/i/Topic/992277653804341_144133903_PERSPECTIVE_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Topic
https://w3id.org/eb/i

In [23]:
for s,p,o in g2.triples((None, None, Literal('HORSEMANSHIP', datatype=XSD.string))):
    print(s,p,o)

In [28]:
for s,p,o in g2.triples((term, None, None)):
    print(s,p,o)

https://w3id.org/eb/i/Article/992277653804341_144133903_ZYGOPHYLLUM_1 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/eb#Article
https://w3id.org/eb/i/Article/992277653804341_144133903_ZYGOPHYLLUM_1 https://w3id.org/eb#numberOfWords 51
https://w3id.org/eb/i/Article/992277653804341_144133903_ZYGOPHYLLUM_1 https://w3id.org/eb#endsAt https://w3id.org/eb/i/Page/992277653804341_144133903_867
https://w3id.org/eb/i/Article/992277653804341_144133903_ZYGOPHYLLUM_1 https://w3id.org/eb#startsAt https://w3id.org/eb/i/Page/992277653804341_144133903_867
https://w3id.org/eb/i/Article/992277653804341_144133903_ZYGOPHYLLUM_1 https://w3id.org/eb#positionPage 26
https://w3id.org/eb/i/Article/992277653804341_144133903_ZYGOPHYLLUM_1 https://w3id.org/eb#name https://w3id.org/eb/i/Article/992277653804341_144133903_ZYGOPHYLLUM_1
https://w3id.org/eb/i/Article/992277653804341_144133903_ZYGOPHYLLUM_1 https://w3id.org/eb#definition in botany, a genus of the decandria monogyma class. The corolla c